In [ ]:
%%capture
!pip install openpyxl
!pip install xlrd
!pip install -q efficientnet
import efficientnet.tfkeras as efn

In [ ]:


import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob as gb


import numpy as np
import pandas as pd
import os
import random, re, math, time
random.seed(a=128)

from os.path import join 

import tensorflow as tf
import tensorflow.keras.backend as K
#import tensorflow_addons as tfa
import efficientnet.tfkeras as efn

from tqdm.keras import TqdmCallback

from PIL import Image
import PIL

import matplotlib.pyplot as plt

from sklearn.model_selection import KFold

from sklearn.utils.class_weight import compute_class_weight

import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from pandas_summary import DataFrameSummary

from kaggle_datasets import KaggleDatasets

from tqdm import tqdm


DEVICE="TPU"
if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    
print("REPLICAS: ", strategy.num_replicas_in_sync)
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array
dataset_dir = "/kaggle/input/ocular-disease-recognition-odir5k/preprocessed_images/"
image_size=224
labels = []
dataset = []
def create_dataset(image_category,label):
    for img in tqdm(image_category):
        image_path = os.path.join(dataset_dir,img)
        try:
            image = cv2.imread(image_path,cv2.IMREAD_COLOR)
            image = cv2.resize(image,(image_size,image_size))
        except:
            continue
        
        dataset.append([np.array(image),np.array(label)])
    random.shuffle(dataset)
    return dataset

In [ ]:
DATA_PATH = '../input/ocular-disease-recognition-odir5k/full_df.csv'
IMG_DIR = '/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Training Images/'

In [ ]:
main_df = pd.read_csv('../input/ocular-disease-recognition-odir5k/full_df.csv')


In [ ]:

IMG_SIZE = 512

def crop(image): 
    # Remove vertical black borders (the image must be already normalized)
    sums = image.sum(axis=0)
    sums = sums.sum(axis=1)
    filter_arr = []
    for s in sums:
        if s == 0:
            filter_arr.append(False)
        else:
            filter_arr.append(True)
    image = image[:, filter_arr]
    
    # Crop to a square shape
    h = image.shape[0]
    w = image.shape[1]    
    
    if h < w:
        x = (w - h)//2
        image = image[:, x:x+h, :]        
    elif h > w:
        x = (h - w)//2
        image = image[x:x+w, :, :]           
    else:
        pass
    
    return image
            

In [ ]:
def preprocess_image(file_name):
    image = cv2.imread(os.path.join(IMG_DIR, file_name))
    
    norm_img = np.zeros(image.shape)
    norm_img = cv2.normalize(image,  norm_img, 0, 255, cv2.NORM_MINMAX)
    
    image = crop(norm_img)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    
    return image

def preprocess_patient(patient_id):
    left_eye_file = str(patient_id) + '_left.jpg'
    right_eye_file = str(patient_id) + '_right.jpg'
    image = cv2.hconcat([preprocess_image(left_eye_file), preprocess_image(right_eye_file)]) 
    return image

# example

In [ ]:
patient_id = main_df.iloc[14]['ID']
image = preprocess_patient(patient_id)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.shape)
plt.imshow(image)


In [ ]:
# example
patient_id = main_df.iloc[7]['ID']
image = preprocess_patient(patient_id)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

In [ ]:
def get_data():
    
    X=[]

    for i in range(main_df.shape[0]):

        patient_id = main_df.iloc[i]['ID']
        image = preprocess_patient(patient_id)
        X.append(image)
        if i==(main_df.shape[0])-1:

            end='\n'
        else:
            end='\r'
        print('processed {}/{}'.format(i+1,(main_df.shape[0])),end=end)
    X=np.array(X)  
    return X  

In [ ]:
X_train=get_data()

In [ ]:
def plot(cmap='gray'):
    plt.figure(figsize=(25,25))
    for n,i in enumerate(list(np.random.randint(0,len(X_train),15))):
        plt.subplot(5,3,n+1)
        plt.imshow(X_train[i],cmap=cmap)
        plt.axis('off')
        plt.title(main_df.iloc[n]['Left-Diagnostic Keywords']+' \n  '+main_df.iloc[i]['Right-Diagnostic Keywords'])

In [ ]:
plot()

In [ ]:
df = pd.read_csv("../input/ocular-disease-recognition-odir5k/full_df.csv")
df.head(500)

In [ ]:
df['labels'].unique()

In [ ]:
main_df=df.copy()

In [ ]:
target = pd.Series(dtype='object')
# this code iterates through every row in the dataframe and creates a list which contains the correct values for the target, when reading the actual columns
# then appends this list to a series which contains the target values
for i in range(main_df.shape[0]):
    target_list = []
    for col in ['N','D','G','C','A','H','M','O']:
        if (main_df.loc[i][col] == 1):
            target_list.append(1)
        else:
            target_list.append(0)
    d = {'target':target_list}
    input_df_row = pd.Series(data=d)
    target = target.append(input_df_row)
target.index = main_df.index

In [ ]:
target1=pd.DataFrame(target)
target1=np.unique(target1)
type(target1)
value=[]
for i in range(len(target1)):
    value.append(target1[i])

In [ ]:
new_data=pd.DataFrame(value)

In [ ]:
data_y=main_df[['N','D','G','C','A','H','M','O']]

data_y=pd.DataFrame(data_y)
data_y_prime=np.array(data_y)

In [ ]:
data=main_df.copy()

In [ ]:
val=[]
for i in range(len(new_data)):
    val.append(i+1)

In [ ]:
value=np.array(value)

In [ ]:
data_y_prime=np.array(data_y)

In [ ]:
print(len(data_y_prime))
print(len(target1))
print(type(data_y_prime))
print(type(target1))

In [ ]:
val=[]
for i in range(len(target1)):
    cas='case'+str(i+1)
    val.append(cas)

In [ ]:
y_val=[]
for i in range(len(data_y_prime)):
    
    for j in range(len(target1)):
        
        if (data_y_prime[i][0]==target1[j][0]) and((data_y_prime[i][1]==target1[j][1])) and (((data_y_prime[i][2]==target1[j][2]))) and ((data_y_prime[i][3]==target1[j][3])) and ((data_y_prime[i][4]==target1[j][4])) and(data_y_prime[i][5]==target1[j][5]) and((data_y_prime[i][6]==target1[j][6])) and (((data_y_prime[i][7]==target1[j][7]))):
            
            if j==0:
                y_val.append(val[0])
            if j==1:
                y_val.append(val[1])
            if j==2:
                y_val.append(val[2]) 
             
            if j==3:
                y_val.append(val[3])
            if j==4:
                y_val.append(val[4])
            if j==5:
                y_val.append(val[5])
             
            
            if j==6:
                y_val.append(val[6])
            if j==7:
                y_val.append(val[7])
            if j==8:
                y_val.append(val[8]) 
             
            if j==9:
                y_val.append(val[9])
            if j==10:
                y_val.append(val[10])
            if j==11:
                y_val.append(val[11])
              
            if j==12:
                y_val.append(val[12])
            if j==13:
                y_val.append(val[13])
            if j==14:
                y_val.append(val[14]) 
             
            if j==15:
                y_val.append(val[15])
            if j==16:
                y_val.append(val[16])
            if j==17:
                y_val.append(val[17])
             
            
            if j==18:
                y_val.append(val[18])
            if j==19:
                y_val.append(val[18])
            if j==20:
                y_val.append(val[20]) 
             
            if j==21:
                y_val.append(val[21])
            if j==22:
                y_val.append(val[22])
            if j==23:
                y_val.append(val[23])  
                
            if j==24:
                y_val.append(val[24])
            if j==25:
                y_val.append(val[25])
            if j==26:
                y_val.append(val[26]) 
             
            if j==27:
                y_val.append(val[27])
            if j==28:
                y_val.append(val[28])
            if j==29:
                y_val.append(val[29])
             
            
            if j==30:
                y_val.append(val[30])
            if j==31:
                y_val.append(val[31])
            if j==32:
                y_val.append(val[32]) 
             
            if j==33:
                y_val.append(val[33])
            if j==34:
                y_val.append(val[34])
            if j==35:
                y_val.append(val[35])   
            if j==36:
                y_val.append(val[36])
            if j==37:
                y_val.append(val[37])     
                
                
        
        
print(len(y_val))      

In [ ]:
df1=main_df.copy()

In [ ]:
y_val=pd.DataFrame(y_val)

In [ ]:
df1['case']=y_val

In [ ]:
df1

In [ ]:
df1.shape

In [ ]:
c=df1['case'].value_counts()

In [ ]:

c=pd.DataFrame(c)
df1['case'].value_counts().index


In [ ]:
df1['case'].value_counts()

In [ ]:
(df1['O']==1).value_counts()

In [ ]:
(df1['N']==1).value_counts()

In [ ]:
(df1['D']==1).value_counts()

In [ ]:
(df1['G']==1).value_counts()

In [ ]:
(df1['C']==1).value_counts()

In [ ]:
(df1['A']==1).value_counts()

In [ ]:
(df1['M']==1).value_counts()

In [ ]:
(df1['H']==1).value_counts()

In [ ]:
import pandas as pd
s1 = pd.Series(['a', 'b', 'a', 'c', 'a', 'b'])
s2 = pd.Series(['a', 'f', 'a', 'd', 'a', 'f', 'f'])
d = pd.DataFrame({'s1': s1, 's2': s2})
d

In [ ]:
d.apply(pd.value_counts).plot(kind='bar', subplots=True)

In [ ]:
df['labels'].apply(pd.value_counts).plot(kind='bar', subplots=True)

In [ ]:
df = pd.read_csv("../input/ocular-disease-recognition-odir5k/full_df.csv")
df.head()

In [ ]:
df['labels'].value_counts().plot(kind='bar', subplots=True)

In [ ]:
df['labels'].value_counts().index

In [ ]:
import seaborn as sns
new_data=df.copy()

print(df['labels'].value_counts())
sns.countplot(x='labels',data=df,palette='Set2')

In [ ]:
new_data['labels']

In [ ]:
new_data=new_data.replace(to_replace =["['N']","['D']","['G']","['C']", "['A']","['H']","['M']","['O']"],
                            value =[0,1,2,3,4,5,6,7])

In [ ]:
new_data.drop(columns=['Left-Fundus','Right-Fundus','Left-Diagnostic Keywords','Right-Diagnostic Keywords','filepath','filename','target','N', 'D' ,'G', 'C', 'A','H', 'M' ,'O'],inplace=True)

In [ ]:
new_data=new_data.set_index('ID')


In [ ]:
new_data.head()

In [ ]:
dum=pd.get_dummies(new_data['Patient Sex'])

In [ ]:
new_data=pd.concat([new_data,dum],axis=1)

In [ ]:
new_data.head()

In [ ]:
new_data.drop(columns=['Patient Sex','Female'],inplace=True)

In [ ]:
new_data

In [ ]:
new_data['Patient Age'].plot('bar')

In [ ]:
def normalize_age(typ=False):
    if typ=='Std':
        age= new_data['Patient Age']-new_data['Patient Age'].mean()/new_data['Patient Age'].std()
    
    if typ=='norm':
        
        age=new_data['Patient Age']/new_data['Patient Age'].max()
    return age
age=normalize_age(typ='norm')
new_data['Age']=age

In [ ]:
new_data

In [ ]:
import seaborn as sns

sns.distplot(new_data['Age'], hist=False)

In [ ]:
sns.distplot(new_data['Patient Age'], hist=False)

In [ ]:
new_data.drop(columns=['Patient Age'],inplace=True)

In [ ]:
new_data

In [ ]:
new_data['labels'].value_counts().plot.pie(autopct="%.1f%%");

In [ ]:
pie, ax = plt.subplots(figsize=[10,8])
labels = new_data['labels'].value_counts().keys()
plt.pie(x=new_data['labels'].value_counts(), autopct="%.1f%%", labels=labels, pctdistance=0.5)
plt.title("Delivery Tips by type", fontsize=14);

In [ ]:
new_data['labels'].value_counts().keys()

In [ ]:
new_data

In [ ]:
plt.imshow(X_train[0])

In [ ]:
len(X_train)
len(new_data)

In [ ]:
y_labels=new_data['labels']
y_labels_cat=df['labels']
plt.imshow(X_train[50])
plt.title(y_labels_cat[50])
plt.figure(figsize=(10,10))
for n,i in enumerate(list(np.random.randint(0,len(X_train),15))):
    plt.subplot(5,3,n+1)
    plt.imshow(X_train[i])
    plt.axis('off')
    plt.title(y_labels_cat[i])
        

In [ ]:
indices=list(range(len(X_train)))
np.random.seed(42)
np.random.shuffle(indices)

ind=int(len(indices)*0.80)
# train data
X_train=X_train[indices[:ind]] 

# validation data


In [ ]:
X_val=X_train_all[indices[-(len(indices)-ind):]] 
y_val=y_train_all[indices[-(len(indices)-ind):]]

# Model 

In [ ]:

from tensorflow.keras.applications.vgg19 import VGG19
vgg = VGG19(weights="imagenet",include_top = False,input_shape=(X_train[0].shape))

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:


IMG_SHAPE = (512,512,3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')

